In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!unrar x -puud "./drive/MyDrive/datasets/divar_dataset.rar" "./drive/MyDrive/datasets//Data-Mining/divar2"

# Initial Tasks

**Importing the Libraries that will be used**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize, LabelEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn import ensemble

In [134]:
np.set_printoptions(precision=2)

Reading the Databases from the uploaded 


In [3]:
digikala_orders = pd.read_csv(r"./drive/MyDrive/datasets/Data-Mining/digikala/orders.csv")
divar_posts = pd.read_csv(r"./drive/MyDrive/datasets/Data-Mining/divar/divar_posts_dataset.csv")

In [4]:
print(digikala_orders.head())
print("\n------\n")
print(divar_posts.head())

   ID_Order  ID_Customer  ...  city_name_fa Quantity_item
0   2714054       469662  ...    محمود آباد           1.0
1  11104039      3063877  ...        خرمدره           1.0
2   4228130      3184893  ...          قرچک           1.0
3  22225624      6888562  ...            قم           1.0
4   4068771      2533490  ...         تهران           1.0

[5 rows x 7 columns]

------

   Unnamed: 0  archive_by_user  ... type year
0           0            False  ...  NaN  NaN
1           1            False  ...  NaN  NaN
2           2            False  ...  NaN  NaN
3           3            False  ...  NaN  NaN
4           4            False  ...  NaN  NaN

[5 rows x 17 columns]


In [5]:
print(digikala_orders.info())
print("\n------\n")
print(divar_posts.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   ID_Order               200000 non-null  int64  
 1   ID_Customer            200000 non-null  int64  
 2   ID_Item                200000 non-null  int64  
 3   DateTime_CartFinalize  200000 non-null  object 
 4   Amount_Gross_Order     200000 non-null  float64
 5   city_name_fa           200000 non-null  object 
 6   Quantity_item          200000 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 10.7+ MB
None

------

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 947635 entries, 0 to 947634
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Unnamed: 0       947635 non-null  int64  
 1   archive_by_user  947635 non-null  bool   
 2   brand            190892 non-null

In [6]:
'''all cats + brand form the item_id'''
# divar_posts["cat1"] = divar_posts["cat1"].astype(str)
# divar_posts["cat2"] = divar_posts["cat2"].astype(str)
# divar_posts["cat3"] = divar_posts["cat3"].astype(str)
# divar_posts["brand"] = divar_posts["brand"].astype(str)
# divar_posts[['brand','cat1', 'cat2','cat3']] = np.sort(divar_posts[['brand','cat1', 'cat2','cat3']], axis=1)

# divar_posts["item_id2"] = divar_posts.groupby(by=['brand','cat3', 'cat2','cat1']).ngroup().astype(str) #add item_id column

'''OR USE BELOW CODE IN ORDER TO DO IT IN A SINGLE LINE'''
divar_posts["item_id"] = divar_posts.set_index(['brand','cat1', 'cat2','cat3']).index.factorize()[0]+1

In [161]:
divar_posts[50:120]

,Unnamed: 0,archive_by_user,brand,cat1,cat2,cat3,city,created_at,desc,id,image_count,mileage,platform,price,title,type,year,item_id
50,50,False,NaN,leisure-hobbies,musical-instruments,guitar-bass-amplifier,Tehran,Thursday 06PM,گیتار همراه با کاور و ۱ عدد پیک صدا و کیفیت خو...,63238507609757,1,NaN,mobile,105000,گیتار با کاور,NaN,NaN,34
51,51,False,NaN,for-the-home,furniture-and-home-decore,beds-bedroom,Shiraz,Thursday 06PM,دو عدد تخت یک نفره + کمــد \nتـمــام MDF \n...,67048359300399,3,NaN,mobile,1000000,سِت تخت و کـمد تمام MDF,NaN,NaN,35
52,52,False,NaN,for-the-home,furniture-and-home-decore,carpets,Tabriz,Thursday 07PM,فرش شش متری طرح هریس. رنگ 16رج 580,33936942805141,2,NaN,mobile,3300000,فرش طرح هریس,NaN,NaN,36
53,53,False,NaN,personal,health-beauty,NaN,Mashhad,Thursday 07PM,کلاه گیس بلند,13961733606436,1,NaN,mobile,20000,کلاه گیس بلند,NaN,NaN,37
54,54,False,NaN,for-the-home,furniture-and-home-decore,sofa-armchair,Qom,Thursday 09PM,یک جفت کوسن مخملی توپُر با اندازه ٤٠در٤٠ سانت(...,53623317617468,2,NaN,mobile,30000,کوسن مخملی توپُر,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,115,False,Apple::اپل,electronic-devices,mobile-tablet,mobile-phones,Tehran,Sunday 01PM,ایفون بسیار تمیز \nاز هر لحاظ سالم,51909812992982,0,NaN,web,1700000,ایفون ٦ (٦٤),NaN,NaN,14
116,116,False,NaN,for-the-home,furniture-and-home-decore,storage,Tehran,Sunday 01PM,نداشتن جا چون چند تا دارم,45088988147807,1,NaN,mobile,85000,جا کفشی نو,NaN,NaN,53
117,117,False,پراید هاچ‌بک::Pride,vehicles,cars,light,Tehran,Sunday 02PM,لاستیک پهن٦٠درصد ١لکه رنگ سالم,32047819627217,0,111.0,web,15500000,پراید ١١١نوک مدادی,NaN,1391,38
118,118,False,NaN,for-the-home,furniture-and-home-decore,beds-bedroom,Mashhad,Sunday 05PM,کم استفاده شده تمیز و سالم,55212899710700,2,NaN,mobile,130000,سرویس رو تخت خوابی,NaN,NaN,35


In [168]:
divar_posts.loc[divar_posts['cat2']=='cars']

,Unnamed: 0,archive_by_user,brand,cat1,cat2,cat3,city,created_at,desc,id,image_count,mileage,platform,price,title,type,year,item_id
2,2,False,NaN,vehicles,cars,heavy,Mashhad,Tuesday 07PM,سریال 43j$NUMبدون شکستگی سه حلقه لاستیک نو یک ...,63194439667302,4,NaN,mobile,-1,لودر کاتر پیلار 950,NaN,NaN,3
8,8,False,سایر,vehicles,cars,light,Tehran,Wednesday 04AM,دوگانه ال پی جی بیمه با تخفیف دارد \nتودوزی نو...,59878876364899,3,180000.0,mobile,-1,پاترول شش سیلندر چهار درب,NaN,1366,7
18,18,False,پژو ۴۰۵::Peugeot 405,vehicles,cars,light,Karaj,Wednesday 11AM,پژو 2000فنی تازه تعمیر بدنه بدون خطو خش تخیف ب...,51728781836074,0,290000.0,web,8500000,پژو مدل 81 مشکی,NaN,1381,13
22,22,False,نیسان::Nissan,vehicles,cars,light,Tehran,Wednesday 01PM,سلام.\nماشین کاملا سالم همه چی فابریک متور زد ...,26591305369165,3,175000.0,mobile,19500000,پاترول 2در,NaN,1372,15
36,36,False,سمند::Samand,vehicles,cars,light,Tehran,Thursday 07AM,سمند ال ایکس بدون رنگ تمامی سرویس ها انجام شده...,35066474998268,4,80000.0,web,23900000,سمند دوگانه کارخانه,NaN,1391,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947612,947612,True,NaN,vehicles,cars,heavy,Tehran,Wednesday 03PM,بادسان فوق بیرنگ سفید رنگ بیمه تابرج 3.96 تخفی...,21866530196097,2,NaN,mobile,48000000,بادسان 8b14 مدل83,NaN,NaN,3
947614,947614,True,NaN,vehicles,cars,heavy,Isfahan,Wednesday 06PM,باسلام .تراکتورازصفری دست خودم بوده.سالم.ودارا...,44067752978166,4,NaN,mobile,-1,تراکتورگلدونی ۲۳۸ایتالیا,NaN,NaN,3
947615,947615,True,سمند::Samand,vehicles,cars,light,Shiraz,Wednesday 06PM,ماشینم تصادفیه با خریدار توافق میکنم سر قیمتش,32248935135739,3,20000.0,web,-1,خودرو سمند تصادفی مدل92,NaN,1392,25
947625,947625,True,پراید صندوق‌دار::Pride,vehicles,cars,light,Karaj,Thursday 03PM,سند۲ برگ،بیمه تا برج$NUMتخفیفات بیمه 15 سال،مع...,30197850752025,4,123000.0,mobile,6900000,پراید 79 بسیار سالم,NaN,1379,29


In [11]:
len(pd.unique(divar_posts['item_id']))

161

# **predicting prices for items**

In [12]:
divar_filtered = divar_posts.filter(["brand","cat1","cat2","cat3", "price"])



In [13]:
divar_filtered.shape

(947635, 5)

remove datapoints where we don't have the price

In [14]:
divar_filtered = divar_filtered.loc[divar_filtered['price']!=-1]

In [16]:
data = pd.get_dummies(divar_filtered, columns=["brand","cat1","cat2","cat3"])
data.head()

,price,brand_Acer::ایسر,brand_Alienware::ایلین‌ویر,brand_Amazon::آمازون,brand_Apple::اپل,brand_Asus::ایسوس,brand_BlackBerry::بلک‌بری,brand_Compaq::کامپک,brand_Dell::دل,brand_Farassoo::فراسو,brand_Fujitsu::فوجیتسو,brand_Gigabyte::گیگابایت,brand_HP::اچ‌پی,brand_HTC::اچ‌تی‌سی,brand_Huawei::هوآوی,brand_LG::ال‌جی,brand_Lenovo::لنوو,brand_MSI::ام‌اس‌آی,brand_Motorola::موتورلا,brand_Nokia::نوکیا,brand_Samsung::سامسونگ,brand_Sony Ericsson::سونی اریکسون,brand_Sony::سونی,brand_Suzuki::سوزوکی,brand_Toshiba::توشیبا,brand_ViewSonic::ویوسونیک,brand_ZTE::زدتی‌ای,brand_ام‌وی‌ام::MVM,brand_بنز::Mercedes-Benz,brand_بی‌ام‌و::BMW,brand_تندر ۹۰::Tondar 90,brand_تویوتا::Toyota,brand_تیبا::Tiba,brand_دوو::Daewoo,brand_رانا::Runna,brand_رنو::Renault,brand_زانتیا::Citroen Xantia,brand_سایر,brand_سمند::Samand,brand_غیره,...,cat3_heavy,cat3_historical-objects,cat3_industrial,cat3_instrument-cleaning-tailoring,cat3_jewelry,cat3_kitchen,cat3_laptops,cat3_light,cat3_lighting,cat3_microwave-stove,cat3_mobile-phones,cat3_mobile-tablet-accessories,cat3_modem-and-network-equipment,cat3_mp3-player,cat3_offices,cat3_parts-and-accessories,cat3_personal-toys,cat3_piano-keyboard,cat3_printer-scaner-copier,cat3_repair-tool,cat3_rhinestones,cat3_rodents-rabbits,cat3_shoes-belt-bag,cat3_shop-and-cash,cat3_sofa-armchair,cat3_stereo-surround,cat3_storage,cat3_stove-and-heating,cat3_strollers-and-accessories,cat3_tables-and-chairs,cat3_tablet,cat3_textile-ornaments,cat3_traditional,cat3_training,cat3_tv-and-stereo-furniture,cat3_tv-projector,cat3_video-dvdplayer,cat3_washer-dryer,cat3_watches,cat3_winter-sports
0,3850000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,30000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,600000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,450000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
train1 = data.drop(['price'],axis=1)
labels = data['price']

In [18]:
x_train , x_test , y_train , y_test = train_test_split(train1 , labels , test_size = 0.10,random_state =2)

In [19]:
reg = LinearRegression()

In [20]:
reg.fit(x_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [21]:
reg.score(x_test,y_test)

-2.4788590317567924e+16

In [59]:
clf = ensemble.GradientBoostingRegressor(n_estimators = 12, max_depth = 10, min_samples_split = 2,
          learning_rate = 0.25, loss = 'ls', verbose=1)
#clf = ensemble.RandomForestRegressor(random_state=1,verbose=1)

In [60]:
clf.fit(x_train, y_train)

      Iter       Train Loss   Remaining Time 
         1 170577556438536.9062            1.57m
         2 136238976884101.6562            1.46m
         3 114218099695629.8750            1.36m
         4 99904780517708.5156            1.26m
         5 90625894266088.2969            1.14m
         6 84231546889384.6406            1.02m
         7 80195403936565.7812           54.28s
         8 77269709120880.6875           47.48s
         9 75427358357110.0000           40.75s
        10 74157880179571.1875           34.11s


GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.2, loss='ls', max_depth=10,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=15,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=1, warm_start=False)

In [129]:
clf.score(x_test,y_test)

0.7158981482742953

got score near 0.75 for 24 estimators and learning rate of 0.25 

In [70]:
a = x_train.head(1)

**Predicting prices we input**

first, we need an empty dataframe that has the dummy cols. then we will add to this, which will be the predocted prices

In [169]:
#first, we need an empty dataframe that has the dummy cols. then we will add to this, which will be the predocted prices
inp = data.drop(['price'],axis=1).iloc[0:0]

#first item to predict
idx = int(inp.shape[0])
inp.loc[idx,:]  = 0
inp.loc[idx,:]['brand_پراید هاچ‌بک::Pride','cat1_vehicles','cat2_cars','cat3_light'] =1

#second item
idx = int(inp.shape[0])
inp.loc[idx,:]  = 0
inp.loc[idx,:]['cat1_electronic-devices','cat2_health-beauty','cat3_beds-bedroom'] =1 #this is an item that doesn't exist by itself in the original dataset!


inp

,brand_Acer::ایسر,brand_Alienware::ایلین‌ویر,brand_Amazon::آمازون,brand_Apple::اپل,brand_Asus::ایسوس,brand_BlackBerry::بلک‌بری,brand_Compaq::کامپک,brand_Dell::دل,brand_Farassoo::فراسو,brand_Fujitsu::فوجیتسو,brand_Gigabyte::گیگابایت,brand_HP::اچ‌پی,brand_HTC::اچ‌تی‌سی,brand_Huawei::هوآوی,brand_LG::ال‌جی,brand_Lenovo::لنوو,brand_MSI::ام‌اس‌آی,brand_Motorola::موتورلا,brand_Nokia::نوکیا,brand_Samsung::سامسونگ,brand_Sony Ericsson::سونی اریکسون,brand_Sony::سونی,brand_Suzuki::سوزوکی,brand_Toshiba::توشیبا,brand_ViewSonic::ویوسونیک,brand_ZTE::زدتی‌ای,brand_ام‌وی‌ام::MVM,brand_بنز::Mercedes-Benz,brand_بی‌ام‌و::BMW,brand_تندر ۹۰::Tondar 90,brand_تویوتا::Toyota,brand_تیبا::Tiba,brand_دوو::Daewoo,brand_رانا::Runna,brand_رنو::Renault,brand_زانتیا::Citroen Xantia,brand_سایر,brand_سمند::Samand,brand_غیره,brand_لیفان::Lifan,...,cat3_heavy,cat3_historical-objects,cat3_industrial,cat3_instrument-cleaning-tailoring,cat3_jewelry,cat3_kitchen,cat3_laptops,cat3_light,cat3_lighting,cat3_microwave-stove,cat3_mobile-phones,cat3_mobile-tablet-accessories,cat3_modem-and-network-equipment,cat3_mp3-player,cat3_offices,cat3_parts-and-accessories,cat3_personal-toys,cat3_piano-keyboard,cat3_printer-scaner-copier,cat3_repair-tool,cat3_rhinestones,cat3_rodents-rabbits,cat3_shoes-belt-bag,cat3_shop-and-cash,cat3_sofa-armchair,cat3_stereo-surround,cat3_storage,cat3_stove-and-heating,cat3_strollers-and-accessories,cat3_tables-and-chairs,cat3_tablet,cat3_textile-ornaments,cat3_traditional,cat3_training,cat3_tv-and-stereo-furniture,cat3_tv-projector,cat3_video-dvdplayer,cat3_washer-dryer,cat3_watches,cat3_winter-sports
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
clf.predict(inp.astype(int))

array([14698608.69,   465828.36])